# 파라메터 서칭
tree base 모델은 설정 가능한 파라메터의 조합에 따라 모델 예측력 차이가 큰 특징을 가지고 있습니다.  
특히, Xgboost 모델의 경우 파라메터 설정에 따른 모델 예측력 차이가 굉장히 크기에 꼭 파라메터 서칭을 진행해주셔야 합니다.  
간단한 문법을 통해 파라메터 서칭을 진행 해보겠습니다.

## 파이썬 기본 문법으로 파라메터 서칭

In [ ]:
from itertools import product
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# iris
iris = load_iris()
y = iris['target']
X = iris['data']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
# 학습데이터에서 10% 비율을 검증셋으로 사용하는 작업
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)

In [ ]:
# 필요 모듈 import
depth = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
est = [100, 300, 500, 700, 900]

best_acc = 0
best_param = 0

for param in list(product(depth, est)):
    print(f'현재 {param} 서칭중')
    model = RandomForestClassifier(random_state=42, max_depth=param[0], n_estimators=param[1])
    model.fit(X_train2, y_train2)
    pred = model.predict(X_val) # 파라메터 서칭을 위한 데이터셋은 검증셋을 사용함
    acc = accuracy_score(y_val, pred)
    if acc > best_acc:
        print('스코어 갱신!! 베스트 파라메터를 업데이트 합니다.')
        best_acc = acc
        best_param = param

print('파라메터 서칭 완료!')
print(f'최적 파라메터는 {best_param}입니다.')

In [ ]:
best_model = RandomForestClassifier(random_state=42, max_depth=best_param[0], n_estimators=best_param[1])
best_model.fit(X_train, y_train)
best_pred = best_model.predict(X_test)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, best_pred))

In [ ]:
!pip install xgboost

In [ ]:
# 데이터셋 로딩


In [ ]:
# 타겟 데이터 분할


In [ ]:
# 학습 데이터 분할


In [ ]:
# 검증 데이터 분할


In [ ]:
# 파라메터 서칭


In [ ]:
# 위 파라메터 서칭 코드로 찾은 최적 모델의 평가지표와 파라메터 확인


In [ ]:
# 최적 모델로 모델 다시 학습 및 평가


## sklearn GridSearchCV
sklearn 패키지에는 위의 파라메터 서칭 과정을 간편하게 진행 할 수 있도록 GridSearchCV 방법론을 제공합니다.  
기존 파라메터 서칭과 함께 cross validation 과정을 추가하여 데이터 분할에 강건한 모델을 선택할 수 있도록 제작 되었습니다.

In [ ]:
''' xgboost 파라메터 서칭 목록
max_depth : classification [3, 5, 7, 9], regression [7, 9, 11, 13, 15]
n_estimators : 부스팅 트리 갯수, [100, 300, 500, 700, 1000, 2000, 3000]
learning_rate : round별 학습률 [0.001, 0.003, 0.01, 0.03]
subsample : 부트스트랩 샘플 비율 [0.6, 0.7, 0.8]
colsample_bytree : 부트스트랩 컬럼 비율 [0.6, 0.7, 0.8]
설명변수 갯수가 많은 경우 아래 파라메터도 서칭
reg_alpha : L1, lasso [1, 3, 5, 7, 9]
reg_lambda : L2, ridge, [1, 3, 5, 7, 9]
'''

In [ ]:
import pandas as pd
df = pd.read_csv('./data/boston.csv')
df.head()

# 타겟 데이터 분할
y = df['y']
X = df.drop('y', axis=1)

# 테스트셋 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# 그리드 서치 import
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

In [ ]:
# 그리드 서치를 위한 모델 및 파라메터 준비
est = XGBRegressor(random_state=42)
param = {
    'max_depth' : [9, 11, 13, 15],
    'n_estimators' : [100, 300, 500, 700],
    'learning_rate' : [0.003, 0.01, 0.03],
    'subsample' : [0.6, 0.7, 0.8],
    'colsample_bytree' : [0.6, 0.7, 0.8],
}

In [ ]:
# 그리드 서치 모델 생성
grid = GridSearchCV(estimator=est,
                    param_grid=param,
                    scoring='r2',
                    verbose=2,
                    cv=5)
'''
estimator : 모델 딕셔너리
param_grid : 파라메터 딕셔너리
scoring=None : 평가방법
n_jobs=None : 학습에 사용할 컴퓨터 코어 갯수
verbose=0 : 리포트 형식 0, 1, 2

scoring 참고
https://scikit-learn.org/stable/modules/model_evaluation.html
'''

In [ ]:
# grid 학습
grid.fit(X_train, y_train)

In [ ]:
# 최적 모델 및 파라메터 확인
grid.best_score_, grid.best_params_

In [ ]:
# 최적 모델로 모델 다시 학습 및 평가
best_pred = grid.predict(X_test)

In [ ]:
# 평가지표 출력
from sklearn.metrics import r2_score, mean_squared_error
print(r2_score(y_test, best_pred))
print(mean_squared_error(y_test, best_pred, squared=False))

# 디스이즈 컴퍼티션!!

In [ ]:
kospi = pd.read_csv('./data/kospi.csv', encoding='cp949')
kospi.head()

# train_test_split test_size=0.3, random_state=42
# rmse 기준 소수점 4자리까지 작으면 우승